In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
import autoreload
import evotsc
autoreload.reload(evotsc)

In [ ]:
# Population-level constants
genome_size = 10_000
gene_length = 100
intergene = 900
interaction_dist = 2500

In [ ]:
def plot_expr(temporal_expr, fitness, plot_name):
    nb_genes, nb_steps = temporal_expr.shape
    
    #colormap = mpl.cm.get_cmap('viridis', nb_genes)(range(nb_genes))
    
    plt.figure(figsize=(9, 6), dpi=200)
    
    plt.ylim(-0.1, 2.1)
    
    for gene in range(nb_genes):
        linestyle = 'solid' if genes[gene].orientation == 0 else 'dashed'
        plt.plot(temporal_expr[gene, :],
                 linestyle=linestyle,
                 #color=colormap[gene],
                 label=f'Gene {gene}')
    
    plt.grid(linestyle=':')
    plt.xlabel('Time', fontsize='large')
    plt.ylabel('Expression level', fontsize='large')
    
    plt.legend(loc='center right')
    plt.title(f'Fitness: {fitness:.2}')
        
    plt.savefig(plot_name, dpi=300, bbox_inches='tight')

In [ ]:
def switch_1_gene(individual): # Evaluate all the individuals obtained by switching 1 gene of the given individual
    for i_switched_gene in range(len(genes)):
        # Switch the current gene 
        to_switch = individual.genes[i_switched_gene]
        to_switch.orientation = 1 - to_switch.orientation

        # Run the switched system and plot
        switch_res, switch_fit = individual.evaluate()
        print(f'Switched gene {i_switched_gene}, fitness: {switch_fit}')
        plot_expr(switch_res, switch_fit, f'switched_{i_switched_gene}')

        # Reset the gene to its former orientation
        to_switch.orientation = 1 - to_switch.orientation

In [ ]:
genes = evotsc.Gene.generate(gene_length=gene_length, intergene=intergene, nb_genes=10)

In [ ]:
zero_indiv = evotsc.Individual(genes=genes,
                               interaction_dist=interaction_dist,
                               interaction_coef=0.0,
                               nb_eval_steps=16)

In [ ]:
plot_expr(*zero_indiv.evaluate(), "interaction coef 0.0.png")

In [ ]:
for i in np.linspace(0.1, 1, 10):
    indiv = evotsc.Individual(genes=genes,
                               interaction_dist=interaction_dist,
                               interaction_coef=i,
                               nb_eval_steps=int(16/i))
    res, fit = indiv.evaluate()
    plot_expr(res, fit, f'interaction coef {i:.1}.png')

In [ ]:
mutation = evotsc.Mutation(intergene_mutation_prob=0.1, intergene_mutation_var=50)

In [ ]:
population = evotsc.Population(init_indiv=init_indiv, nb_indivs=1000, mutation=mutation)

In [ ]:
%%timeit
population.evaluate()

In [ ]:
np.mean([indiv.fitness for indiv in population.individuals])

In [ ]:
%%timeit
population.evolve(nb_steps=100)

In [ ]:
import cProfile

In [ ]:
cProfile.run('population.evolve(nb_steps=100)', 'stats_loop_builtinmax.cprof')

In [ ]:
best_indivs = population.best_indivs

In [ ]:
orig_expr, orig_fit = best_indivs[0].evaluate()

In [ ]:
final_expr, final_fit = best_indivs[-1].evaluate()

In [ ]:
plot_expr(orig_expr, orig_fit, 'original individual')

In [ ]:
plot_expr(final_expr, final_fit, 'final individual')

In [ ]:
np.maximum(0.4, 0.5)